# FINAL PROJECT

### DATA ENGINEERING (MG-GY 8411)
##### Prof. Carlos De Oliveira
"SELL USDCHF"

- __Name: Shikhar Sharma__ 
- __Net Id: ss16023__

1- ___Import Libraries___

In [ ]:
# Importing important libraries
from metaapi_cloud_sdk import MetaApi
import asyncio
import time
from datetime import datetime, timedelta
import requests
import csv
import datetime
from pymongo import MongoClient
import pytz

2- ___Making Connection to MT4 using MT4 API___

- _This code imports the MetaApi library and initializes a MetaApi object using an API token._ 
- _It then defines the accountId variable representing the ID of a MetaTrader account._ 
- _Finally, it creates a connection to the MetaApi server using the provided API token, which can then be used to interact with the MetaTrader platform via the MetaApi library._

In [2]:
token = "3ZVfdBqX1x1HZbhkyH1uucseMVuvmuXRSx5dB5MxA7JwJSV4H4rTgU4BHqb8frmk"
metaapi = MetaApi(token=token)
accountId = '4e0fcf75-67c5-40af-aace-fcc246f9c860'
api = MetaApi(token)

3- ___Connecting to MongoDb Database___

- _This code creates a connection to a MongoDB database using the MongoClient._
- _It then creates a database called "Final_Db" using the client_mongo object and a collection called "hw6" using the collection object._
- _Finally, it creates another collection called "collection2" using the collection2 object._

In [3]:
client_mongo = MongoClient('mongodb://localhost:27017/')
db1 = client_mongo['Final_Db']
collection = db1['hw6']
collection2 = db1['collection2']

### First interval
4- ___Initiating the first interval for EURUSD currency pair from 3 pm to 5 pm___

In [4]:
# Define an asynchronous function called "first_interval" with two parameters: "trade_type" and "symbol"
async def first_interval(trade_type, symbol,connection,start,end):
    
    # Initialize "average" and "price_list" variables
    average = 0
    price_list = []
    ny_timezone = pytz.timezone('America/New_York')
    # Check if the current hour is between 3pm and 5pm
    while datetime.datetime.now(ny_timezone).hour != start:
        print(f"Waiting for current hour to become {start}...")
        time.sleep(60)
    # Create an infinite loop
    while True:
        
        # Get the current hour
        now = datetime.datetime.now(ny_timezone).hour
            
        if (now < end):
            try:
                # Get the current price for the specified symbol
                price = await connection.get_symbol_price(symbol)

                # Check if the price is a dictionary and if it has an 'ask' key
                if isinstance(price, dict) and 'ask' in price:
                    price = price['ask']

                # Check if the price is None
                if price is None:
                    print('Failed to get symbol price')
                    continue
                
                # Set slippage and comment variables
                slippage = 10
                comment = 'Test order'
                
                # Set volume to 0.10
                vol = 0.10

                # Print the current price and append it to the price_list
                print(price)
                price_list.append(price)
                
                # If trade_type is "BUY", create a market buy order
                if trade_type == "BUY":
                   
                    await connection.create_market_buy_order(
                        symbol=symbol, 
                        volume=vol, 
                        stop_loss=0.9, 
                        take_profit=2.0
                    )
                    
                # If trade_type is "SELL", create a market sell order
                elif trade_type == "SELL":
                    
                    await connection.create_market_sell_order(
                        symbol=symbol, 
                        volume=vol, 
                        stop_loss=2.0, 
                        take_profit=0.9
                    )
                        
                else:
                    print("Not able to execute")

                # Print a success message with the order details
                print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
                
                # Wait for 360 seconds before checking the price again
                time.sleep(360)

            except Exception as err:
                # Print an error message if there is an exception while creating the order
                print(f'Failed to create order: {err}')
            
            
           
        
        # If the current hour is not between 3pm and 5pm, break out of the loop
        else:
            break
    
    # Calculate the average of the prices in price_list and store it in "avg_1"
    final_avg = sum(price_list)/len(price_list)
    
    print(final_avg)
    
    # Return a dictionary containing both the average and the "price_list".
    return {"average":final_avg,"price_list":price_list}

5- ___Intitializing final 2,3, and 4 intervals simultaneously from 7pm to 10pm, 12pm to 1am, and 3am to 6am respectively.___

In [ ]:
async def final_intervals(avg,total_units,trade_type,symbol,vol,connection,pl,start,end):
    
    # Setting total units to initial value
    total_units = total_units
    
    # Initializing an empty list to store prices
    price_list = []
    
    # Initializing units remaining to 0
    units_remaining = 0
    
    # Initializing units executed to 0
    units_executed = 0
    
    ny_timezone = pytz.timezone('America/New_York')
    
    # Checking if the current hour is within the trading window
    while datetime.datetime.now(ny_timezone).hour != start:
        print(f"Waiting for current hour to become {start}...")
        time.sleep(60)
    
    # Looping continuously
    while True:
        
        # Getting the current hour
        now_time = datetime.datetime.now(ny_timezone).hour
        
        # Getting the current symbol price
        price = await connection.get_symbol_price(symbol)

        # Checking if the price is in dictionary format and has an 'ask' key
        if isinstance(price, dict) and 'ask' in price:

            # Setting the price to the ask value in the dictionary
            price = price['ask']

        # Checking if price is None
        if price is None:

            # Printing an error message and continuing to the next iteration
            print('Failed to get symbol price')
            continue
            
        if now_time < end and price >=avg:
            try:
                
                    
                # Setting slippage to 10
                slippage = 10
                
                # Setting comment to 'Test order'
                comment = 'Test order'
                
                # Appending the current price to the price list
                price_list.append(price)
                
                # Checking the trade type and price against the average
                if trade_type == "BUY":
                    
                    # Creating a market buy order
                    await connection.create_market_buy_order(symbol=symbol,volume=vol, stop_loss=0.9,take_profit=2.0)
                    
                    # Incrementing the executed units
                    if units_executed < (total_units-(vol*100000)):
                        units_executed += vol * 100000
                        print(f"UNITS EXECUTED {units_executed}")
                    else:
                        units_executed +=vol * 100000
                        print(f"UNITS EXECUTED {units_executed}")
                        units_remaining = total_units - units_executed
                        print(f"UNITS REMAINING {units_remaining}")
                        print("Exiting while loop because all units executed")
                        break
                        
                    # Calculating the remaining units
                    units_remaining = total_units - units_executed
                    print(f"UNITS REMAINING {units_remaining}")
                elif trade_type == "SELL" and price < avg:
                    
                    # Creating a market sell order
                    await connection.create_market_sell_order(symbol=symbol,volume=vol, stop_loss=2.0, take_profit=0.9)
                else:
                    # Printing an error message and breaking the loop
                    print("Exiting while loop because condition failed")
                    break
                    
                # Printing a success message with the trade type, symbol, volume, and price
                print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
            except Exception as err:
                
                # Printing an error message with the error details
                print(f'Failed to create order: {err}')
                
            # Waiting for 6 minutes before checking the price again
            time.sleep(360)
        else:
            # Breaking the loop if the current hour is outside the trading window
            print("Exiting while loop because condition failed")
            units_remaining = total_units - units_executed
            print(f"UNITS REMAINING {units_remaining}")
            break
            
    # Calculating the average price from the price list
    # Adding the calculated average to the initial average
    if len(price_list) !=0:
        new_total_average = sum(pl+price_list)/len(pl+price_list)
    else:
        new_total_average = avg
    
    
    # Printing the total average and the remaining units
    print(f'Total Average: {new_total_average}')
    
    # Returning the remaining units, executed units, and total average for each interval
    return {"non_executed":units_remaining, "executed":units_executed,"new_total_avg":new_total_average,"price_list":price_list}

## Final Execution

6- ___Placing the final 'BUY' or 'SELL' market order based on the certain conditions.___ 

In [5]:
async def after_final(avg,trade_type,symbol,connection,non_executed_units,executed_units):
    
    time.sleep(1800)
    
    remain_units = 0
    
    prev_30 = False
    
    while True:
    
        price = await connection.get_symbol_price(symbol)

            # Checking if the price is in dictionary format and has an 'ask' key
        if isinstance(price, dict) and 'ask' in price:
            # Setting the price to the ask value in the dictionary
            price = price['ask']

        # Checking if price is None
        if price is None:
            # Printing an error message and continuing to the next iteration
            print('Failed to get symbol price')
            continue

        if trade_type == "BUY" and price >=avg:
                try:
                    # Setting slippage to 10
                    slippage = 10

                    # Setting comment to 'Test order'
                    comment = 'Test order'

                    remain_units = int(0.50 * non_executed_units)

                    vol = remain_units/100000

                    # create a loop to execute all trade units by taking 0.10 at a time.

                    # Creating a market buy order
                    await connection.create_market_buy_order(symbol=symbol,volume=vol, stop_loss=0.9,take_profit=2.0)
                    # Printing a success message with the trade type, symbol, volume, and price
                    print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')

                    prev_30 = True
                except Exception as err:
                    # Printing an error message with the error details
                    print(f'Failed to create order: {err}')
                break   

            # Waiting for 6 minutes before checking the price again
        elif trade_type == "SELL" and price <=avg:
                try:
                     # Setting slippage to 10
                    slippage = 10

                    # Setting comment to 'Test order'
                    comment = 'Test order'

                    remain_units = int(0.50 * trade_units)

                    vol = remain_units/100000
                    await connection.create_market_sell_order(symbol=symbol,volume=vol, stop_loss=2.0, take_profit=0.9)
                    # Printing a success message with the trade type, symbol, volume, and price
                    print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
                    prev_30 = True
                except Exception as err:

                    # Printing an error message with the error details
                    print(f'Failed to create order: {err}')
                break
        else:
            print("Did not placed the trade in first 30 minutes")

        remain_units_30 = non_executed_units - remain_units
        print("-----******-----")
        print(remain_units_30)

        time.sleep(1800)

        price = await connection.get_symbol_price(symbol)

            # Checking if the price is in dictionary format and has an 'ask' key
        if isinstance(price, dict) and 'ask' in price:

            # Setting the price to the ask value in the dictionary
            price = price['ask']

        # Checking if price is None
        if price is None:

            # Printing an error message and continuing to the next iteration
            print('Failed to get symbol price')
            continue

        if trade_type == "BUY":
            if price>=avg and prev_30 == True:
                try:
                    # Setting slippage to 10
                    slippage = 10

                    # Setting comment to 'Test order'
                    comment = 'Test order'

                    vol = remain_units_30/100000

                    # Creating a market buy order
                    await connection.create_market_buy_order(symbol=symbol,volume=vol, stop_loss=0.9,take_profit=2.0)
                    # Printing a success message with the trade type, symbol, volume, and price
                    print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
                except Exception as err:
                    # Printing an error message with the error details
                    print(f'Failed to create order: {err}')
                break

            elif price<avg and prev_30 == True:
                try:
                    # Setting slippage to 10
                    slippage = 10

                    # Setting comment to 'Test order'
                    comment = 'Test order'

                    vol = remain_units_30/100000

                    # Creating a market buy order
                    await connection.create_market_buy_order(symbol=symbol,volume=vol, stop_loss=0.9,take_profit=2.0)
                    # Printing a success message with the trade type, symbol, volume, and price
                    print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
                except Exception as err:
                    # Printing an error message with the error details
                    print(f'Failed to create order: {err}')
                break

            elif price<avg and prev_30 == False:

                target_value = int(0.20 * executed_units)
                vol = target_value/100_000/20

                for i in range(1,21):
                    time.sleep(60)

                    try:
                        # Setting slippage to 10
                        slippage = 10

                        # Setting comment to 'Test order'
                        comment = 'Test order'
                        await connection.create_market_sell_order(symbol=symbol,volume=vol, stop_loss=2.0, take_profit=0.9)
                        # Printing a success message with the trade type, symbol, volume, and price
                        print(f'Successfully created {trade_type} order for {vol} {symbol} in {i} interval at {price}')

                    except Exception as err:

                        # Printing an error message with the error details
                        print(f'Failed to create order: {err}')
                break

            else:
                print("Unable to execute")
                break

    # SHORT WHEN trade type is SELL.
        elif trade_type == "SELL":
            if price<=avg and prev_30 == True:
                try:
                    # Setting slippage to 10
                    slippage = 10

                    # Setting comment to 'Test order'
                    comment = 'Test order'

                    vol = remain_units_30/100000

                    # Creating a market buy order
                    await connection.create_market_sell_order(symbol=symbol,volume=vol, stop_loss=0.9,take_profit=2.0)
                    # Printing a success message with the trade type, symbol, volume, and price
                    print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
                except Exception as err:
                    # Printing an error message with the error details
                    print(f'Failed to create order: {err}')
                break

            elif price>avg and prev_30 == True:
                try:
                    # Setting slippage to 10
                    slippage = 10

                    # Setting comment to 'Test order'
                    comment = 'Test order'

                    vol = remain_units_30/100000

                    # Creating a market buy order
                    await connection.create_market_buy_order(symbol=symbol,volume=vol, stop_loss=0.9,take_profit=2.0)
                    # Printing a success message with the trade type, symbol, volume, and price
                    print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')
                except Exception as err:
                    # Printing an error message with the error details
                    print(f'Failed to create order: {err}')
                break

            elif price>avg and prev_30 == False:

                target_value = int(0.20 * executed_units)
                vol = target_value/100_000/20

                for i in range(1,21):
                    time.sleep(60)

                    try:
                        # Setting slippage to 10
                        slippage = 10

                        # Setting comment to 'Test order'
                        comment = 'Test order'
                        await connection.create_market_buy_order(symbol=symbol,volume=vol, stop_loss=2.0, take_profit=0.9)
                        # Printing a success message with the trade type, symbol, volume, and price
                        print(f'Successfully created {trade_type} order for {vol} {symbol} at {price}')


                    except Exception as err:

                        # Printing an error message with the error details
                        print(f'Failed to create order: {err}')
                break

            else:
                print("Unable to execute")
                break
        else:
            print("Trade Type not found")
            break
    
    return {"Executed_units":remain_units_30}
                
            

7- ___Running the "main()" function to execute the code for first, second, third, and fourth interval.___

In [ ]:
async def main():
    
    # create an instance of the MetaApi object
    api = MetaApi(token)
    
    try:
        # get account details
        account = await api.metatrader_account_api.get_account(accountId)
        initial_state = account.state
        deployed_states = ['DEPLOYING', 'DEPLOYED']
        
        # check if account is already deployed and connected to broker

        if initial_state not in deployed_states:
            #  wait until account is deployed and connected to broker
            print('Deploying account')
            await account.deploy()
            
        # wait for the account to be connected to broker
        print('Waiting for API server to connect to broker (may take couple of minutes)')
        await account.wait_connected()

        # get RPC connection to MetaApi API
        connection = account.get_rpc_connection()
        
        # connect to MetaApi API 
        conn = await connection.connect()
        
        # wait for synchronization with MetaTrader
        await connection.wait_synchronized()
        
        # define intervals to perform trading
        intervals = [1,2,3,4]
        
        # initialize variables for tracking executed and non-executed orders, as well as previous averages
        total_executed = 200000
        total_non_executed = 800000
        #prev_averages = 0
        symbol = "USDCHF"
        trade_type = "SELL"
        
        # loop through the defined intervals and execute trades based on certain conditions
        #while True:
            
        for interval in intervals:
                
            print(f"-------*****{interval}*****-------")

            # execute trades based on first interval conditions
            if interval == 1:
                start = 15
                end = 17
                first_intrvl = await first_interval(trade_type,symbol,connection,start,end)
                average_1 = first_intrvl["average"]
                price_list = first_intrvl["price_list"]
                

#                 # execute trades based on second interval conditions
            elif interval == 2:
                vol = 0.10
              #  start_minute=datetime.datetime.now().minute
               # end_minute= datetime.
                start = 19
                end = 22
                second_intrvl = await final_intervals(average_1,300000,trade_type,symbol,vol,connection,price_list,start,end)
                price_list_2 = second_intrvl["price_list"]
                print("*")
                average_2 = second_intrvl["new_total_avg"]
                print("**")
                executed_amount_2 = second_intrvl["executed"]
                print("***")
                non_executed_amount_2 = second_intrvl["non_executed"]
                print("****")
                total_executed +=executed_amount_2
                print("*****")
                total_non_executed -= executed_amount_2
                
               # prev_averages += average_1+average_2

            # execute trades based on third interval conditions
            elif interval == 3:
               # start_hour=23
                #end_hour=1
                start = 23
                end = 1
                total_units_3 = 200_000+non_executed_amount_2
                vol = (total_units_3/20)/1e5
                third_intrvl = await final_intervals(average_2,total_units_3,trade_type,symbol,vol,connection,price_list_2,start,end)
                price_list_3 = third_intrvl["price_list"]
                average_3 = third_intrvl['new_total_avg']
                executed_amount_3 = third_intrvl['executed']
                non_executed_amount_3 = third_intrvl['non_executed']
                total_executed += executed_amount_3
                total_non_executed -=executed_amount_3

#                  # execute trades based on fourth interval conditions
            elif interval == 4:
        
                start = 3
                end = 6
                total_units_4 = 300000+non_executed_amount_3
                vol = (total_units_4/20)/1e5
                fourth_intrvl = await final_intervals(average_3,total_units_4,trade_type,symbol,vol,connection,price_list_3,start,end)
                price_list_4 = fourth_intrvl["price_list"]
                average_4 = fourth_intrvl["new_total_avg"]
                executed_amount_4 = fourth_intrvl['executed']
                non_executed_amount_4 = fourth_intrvl['non_executed']
                total_executed += executed_amount_4
                total_non_executed -=executed_amount_4

            # Break loop if no intervals are left to execute.
            else:

                break
                
        final_run = await after_final(average_4,trade_type,symbol,connection,total_non_executed,total_executed)
        final_exc_remained = final_run["Executed_units"]
                    
        # Set Start and End time to get history orders from last one day to current time.
       
        print(average_1)
        print(average_2)
        print(average_3)
        print(average_4)
        print(executed_amount_2)
        print(non_executed_amount_2)
        print(executed_amount_3)
        print(non_executed_amount_3)
        print(executed_amount_4)
        print(non_executed_amount_4)
        print(total_executed)
        print(total_non_executed)
        print("------********---------")
        print(final_exc_remained)
        start_time = datetime.datetime.now() - timedelta(hours=12)
            
        end_time = datetime.datetime.now()

        # Get history orders
        orders = await connection.get_history_orders_by_time_range(start_time=start_time, end_time=end_time)
#         price = order['openPrice']
#         currency = order['symbol']
#         time = order['time']
#         order_id = order['id']
#         order_type = order["type"]
        
        
        data = {
            "Interval": 1,
            "Executed Amt": 200000,
            "Non Executed Amt": 0,
            "% of Executed Amt": 100,
            "% of Non-Executed Amt": 0,
            "Average Price": average_1,
            "Interval 1": {
                "Start Time": "15:00:00",
                "End Time": "17:00:00",
                "Executed Amt": 200000,
                "Non Executed Amt": 0,
                "% of Executed Amt": 100,
                "% of Non-Executed Amt": 0,
                "Average Price": average_1
            },
            "Interval 2": {
                "Start Time": "19:00:00",
                "End Time": "22:00:00",
                "Executed Amt": executed_amount_2,
                "Non Executed Amt": non_executed_amount_2,
                "% of Executed Amt": (executed_amount_2/300_000)*100,
                "% of Non-Executed Amt": (non_executed_amount_2/300_000)*100,
                "Average Price": average_2
            },
            "Interval 3": {
                "Start Time": "23:00:00",
                "End Time": "01:00:00",
                "Executed Amt": executed_amount_3,
                "Non Executed Amt": non_executed_amount_3,
                "% of Executed Amt": (executed_amount_3/total_units_3)*100,
                "% of Non-Executed Amt": (non_executed_amount_3/total_units_3)*100,
                "Average Price": average_3
            },
            "Interval 4": {
                "Start Time": "03:00:00",
                "End Time": "06:00:00",
                "Executed Amt": executed_amount_4,
                "Non Executed Amt": non_executed_amount_4,
                "% of Executed Amt": (executed_amount_4/total_units_4)*100,
                "% of Non-Executed Amt": (non_executed_amount_4/total_units_4)*100,
                "Average Price": average_4
            },
            "Average of the entire execution": average_4
        }

#          Insert the data into the MongoDB collection
        result = collection.insert_one(data)
        result2 = collection2.insert_one(orders)
        
#          Specify the file name and location
#         filename = 'history_orders_3.csv'

#         # Open the file in write mode
#         with open(filename, 'w', newline='') as file:

#             writer = csv.writer(file)

#             # Write the headers
#             headers = list(orders['historyOrders'][0].keys())
#             writer.writerow(headers)

#             # Write the data
#             for order in orders['historyOrders']:
#                 row = [order.get(key, '') for key in headers]
#                 writer.writerow(row)

#         print('Data has been saved')
        
    except Exception as err:
        print(err)
    
    exit()        
            
#             if ((datetime.now.hour >= 15 && datetime.now.hour < 17) or
#                datetime.now.nour >=19 and datetime.now.hour <22):
                
#hr >= 17 and hr<20:
              
            
            
#if datetime.datetime.now <= "16:59:59":                
#if minu  <= end_time: # run this loop in such a way that it keeps running and adding 6 minutes to start time minutes until the hr i
# the start time becomes equal to end time hr 17
                

                        # calculate order volume based on the hour of the day
#                     await connection.create_market_buy_order(symbol=symbol,
#                                                              volume=vol, 
#                                                              stop_loss=0.9, 
#                                                              take_profit=2.0
#                                                             )
                    
                
            #if (datetime.now.hour >= 19 && datetime.now.hour < 22):
                
                
#                 datetime.now.nour >=19 and datetime.now.hour <22):      
                    # call the function
    
    
                    
                    
                        
             # Add this below two hour interval condition , not here.
                    
                    
           
            
            
            # Update next interval units in the dictionary or something else by adding units remain.
         #   next_interval_units =UPDATE func.units_remaining + dict.third_key_value_pair.units
            

In [ ]:
await main()